In [2]:
import pandas as pd
raw_dataset = pd.read_csv('Multiple Cause of Death 1999-2014 v1.1.csv')

In [3]:
raw_dataset.columns
columns_to_drop = ['Crude Rate','Crude Rate Lower 95% Confidence Interval','Crude Rate Upper 95% Confidence Interval']
cleaned_dataset = raw_dataset.drop(columns=columns_to_drop)

In [12]:
import numpy as np
cleaned_dataset = raw_dataset.replace(['Suppressed', 'Unreliable'], np.nan)
cleaned_dataset = cleaned_dataset.dropna()
print(cleaned_dataset.isin(['Suppressed', 'Unreliable']).any().any())

False


In [18]:
print(cleaned_dataset)
cleaned_dataset['Deaths'] = pd.to_numeric(cleaned_dataset['Deaths'], errors='coerce')
total_deaths_by_state = cleaned_dataset.groupby('State')['Deaths'].sum().sort_values(ascending=True)

       State  Year Deaths  Population Crude Rate  \
0    Alabama  1999     39     4430141        0.9   
1    Alabama  2000     46     4447100          1   
2    Alabama  2001     67     4467634        1.5   
3    Alabama  2002     75     4480089        1.7   
4    Alabama  2003     54     4503491        1.2   
..       ...   ...    ...         ...        ...   
811  Wyoming  2010     49      563626        8.7   
812  Wyoming  2011     47      568158        8.3   
813  Wyoming  2012     47      576412        8.2   
814  Wyoming  2013     52      582658        8.9   
815  Wyoming  2014     57      584153        9.8   

    Crude Rate Lower 95% Confidence Interval  \
0                                        0.6   
1                                        0.8   
2                                        1.2   
3                                        1.3   
4                                        0.9   
..                                       ...   
811                                    

In [19]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error, r2_score, root_mean_squared_error, mean_absolute_percentage_error

cleaned_dataset['Year'] = pd.to_numeric(cleaned_dataset['Year'])
yearly_deaths = cleaned_dataset.groupby('Year')['Deaths'].sum().reset_index()

X = yearly_deaths['Year'].values.reshape(-1, 1)
y = yearly_deaths['Deaths'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

last_year = yearly_deaths['Year'].max()
future_years = np.arange(last_year + 1, last_year + 16).reshape(-1, 1)
predicted_deaths = model.predict(future_years)



mse = mean_squared_error(y_test, model.predict(X_test))
print('Mean Squared Error:', mse)
mae = mean_absolute_error(y_test, model.predict(X_test))
print("Mean Absolute Error:", mae)


predictions_df = pd.DataFrame({'Year': future_years.flatten(), 'Predicted Deaths': predicted_deaths})
predictions_df

Mean Squared Error: 289263.07296205254
Mean Absolute Error: 497.3334837544244


,Year,Predicted Deaths
0,2015,29496.768051
1,2016,30834.118231
2,2017,32171.468412
3,2018,33508.818592
4,2019,34846.168773
5,2020,36183.518953
6,2021,37520.869134
7,2022,38858.219314
8,2023,40195.569495
9,2024,41532.919675


In [20]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error, r2_score, root_mean_squared_error, mean_absolute_percentage_error

def predict_state_deaths(state_name, years_to_predict=15):
    state_data = cleaned_dataset[cleaned_dataset['State'] == state_name]
    if state_data.empty:
        return "State not found in the dataset."

    state_yearly_deaths = state_data.groupby('Year')['Deaths'].sum().reset_index()
    X = state_yearly_deaths['Year'].values.reshape(-1, 1)
    y = state_yearly_deaths['Deaths'].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 42)

    model = LinearRegression()
    model.fit(X_train, y_train)

    last_year = state_yearly_deaths['Year'].max()
    future_years = np.arange(last_year + 1, last_year + 1 + years_to_predict).reshape(-1, 1)
    predicted_deaths = model.predict(future_years)

    predictions_df = pd.DataFrame({'Year': future_years.flatten(), 'Predicted Deaths': predicted_deaths})

    mse = mean_squared_error(y_test, model.predict(X_test))
    print('Mean Squared Error:', mse)
    mae = mean_absolute_error(y_test, model.predict(X_test))
    print("Mean Absolute Error:", mae)
    print('Intercept:', model.intercept_)
    print("Slope:", model.coef_)
    sse = sum((y_test - model.predict(X_test)**2))
    r2 = r2_score(y_true = y_test, y_pred = model.predict(X_test))
    print("r**2:", r2)


    return predictions_df

state_to_predict = "Nevada" #@param {type:"string"}
predictions = predict_state_deaths(state_to_predict)
predictions

Mean Squared Error: 2729.5943564365334
Mean Absolute Error: 43.56498194945925
Intercept: -43481.33574007218
Slope: [21.86101083]
r**2: 0.7719303693993246


,Year,Predicted Deaths
0,2015,568.601083
1,2016,590.462094
2,2017,612.323105
3,2018,634.184116
4,2019,656.045126
5,2020,677.906137
6,2021,699.767148
7,2022,721.628159
8,2023,743.489170
9,2024,765.350181
